# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Study data files
weather_metadata = "output_data/my_weather_data.csv"

# Read the weather data and the study results
weather_data = pd.read_csv(weather_metadata)

# Combine the data into a single dataset
weatherdf = pd.DataFrame(weather_data)
weatherdf.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Lagoa,37.1363,-8.4541,66.80,68,0,8.03,PT,1569736800
1,1,Rikitea,-23.1203,-134.9692,71.66,86,6,10.51,PF,1569736800
2,2,Albany,42.6512,-73.7550,64.40,95,88,4.76,US,1569736800
3,3,Esperance,-33.8583,121.8932,62.44,87,37,18.97,AU,1569736800
4,4,Faanui,-16.4834,-151.7500,78.16,84,19,20.13,PF,1569736800


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
# Store 'Lat' and 'Lng' into  locations 
locations = weatherdf[["Latitude", "Longitude"]].astype(float)

#weatherdf = weatherdf.dropna()

hum_rate = weatherdf["Humidity"].astype(float)

In [31]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacationheatmap.png")
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
#Narrow down the DataFrame to find your ideal weather condition.

#A max temperature lower than 80 degrees but higher than 70.
first = weatherdf.loc[weatherdf["Max Temp"]<80,:]
second = first.loc[weatherdf["Max Temp"]>70,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Wind Speed"]<10,:]
third

#Zero cloudiness.
fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,Byron Bay,-28.6500,153.6167,71.60,63,0,6.08,AU,1569736800
32,32,Busselton,-33.6445,115.3489,74.41,75,0,3.22,AU,1569736800
56,56,Pombia,35.0089,24.8684,75.87,50,0,7.94,GR,1569736800
76,76,Shahreza,32.0116,51.8652,73.13,15,0,6.53,IR,1569736800
103,103,Miranda,-20.2402,-56.3845,75.46,49,0,5.75,BR,1569736800


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Latitude']
hotel_df['Lng'] = fifth['Longitude']
#hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng
24,Byron Bay,AU,-28.6500,153.6167
32,Busselton,AU,-33.6445,115.3489
56,Pombia,GR,35.0089,24.8684
76,Shahreza,IR,32.0116,51.8652
103,Miranda,BR,-20.2402,-56.3845


In [14]:
#without key
#https://maps.googleapis.com/maps/api/place/textsearch/json?query=hotels&location={longitude},{latitude}&radius=5000&key={g_key}

In [15]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 24: Byron Bay.
Closest hotel to Byron Bay at -28.65 , 153.6167 is Bay Motel.
------------
Retrieving Results for Index 32: Busselton.
Closest hotel to Busselton at -33.6445 , 115.3489 is Sojourn on Gale.
------------
Retrieving Results for Index 56: Pombia.
Closest hotel to Pombia at 35.0089 , 24.8684 is Rigakis, El. & M., O.E..
------------
Retrieving Results for Index 76: Shahreza.
Closest hotel to Shahreza at 32.0116 , 51.8652 is Almas System(Shahreza).
------------
Retrieving Results for Index 103: Miranda.
Closest hotel to Miranda at -20.2402 , -56.3845 is Câmara Municipal de Miranda.
------------
Retrieving Results for Index 226: Avera.
Closest hotel to Avera at 33.194 , -82.5271 is Avera Volunteer Fire Department.
------------
Retrieving Results for Index 245: Bako.
Missing field/result... skipping.
------------
Retrieving Results for Index 264: Morondava.
Closest hotel to Morondava at -20.2954 , 44.2798 is Madawatt Morondava.
------------
Retrieving

In [16]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
24,Byron Bay,AU,-28.6500,153.6167,Bay Motel
32,Busselton,AU,-33.6445,115.3489,Sojourn on Gale
56,Pombia,GR,35.0089,24.8684,"Rigakis, El. & M., O.E."
76,Shahreza,IR,32.0116,51.8652,Almas System(Shahreza)
103,Miranda,BR,-20.2402,-56.3845,Câmara Municipal de Miranda


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [30]:

fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("vacationheatmapwithhotels.png")

fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>